<a href="https://colab.research.google.com/github/Ssurf777/pytorch_tensorflow_MLP_compare/blob/main/LLM_and_SurrogateOperator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Generate Python code to download an offline version of Llama, set up a dictionary to map ambiguous terms for "排気側", "吸気側", "フロント側", and "リア側" to standardized keys, and generate a JSON output containing target temperatures for these four directions based on user input.

## Set up the environment

### Subtask:
Install necessary libraries and dependencies for running Llama offline.


**Reasoning**:
The subtask requires installing several libraries. I will use pip to install them in a single code block.



**Reasoning**:
The previous installation failed because 'json' is a built-in Python library and does not need to be installed. I will remove 'json' and retry the installation of the remaining libraries.



In [1]:
%pip install transformers torch ctranslate2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 12.5 MB/s eta 0:00:00


## Download llama

### Subtask:
Download the specified version of Llama for offline use.


**Reasoning**:
Import the necessary classes from the transformers library and specify the model name.



In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

**Reasoning**:
Load the tokenizer and model for offline use, allowing download if not cached locally.



In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=False)
model = AutoModelForCausalLM.from_pretrained(model_name, local_files_only=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Define direction mapping

### Subtask:
Create a dictionary to map ambiguous terms for the four directions (exhaust, intake, front, rear) to standardized keys.


**Reasoning**:
Create a Python dictionary to map ambiguous terms for the four directions to standardized keys as instructed.



In [4]:
direction_mapping = {
    "exhaust": ["排気側", "exhaust"],
    "intake": ["吸気側", "intake"],
    "front": ["フロント側", "front"],
    "rear": ["リア側", "rear"]
}

display(direction_mapping)

{'exhaust': ['排気側', 'exhaust', 'outlet'],
 'intake': ['吸気側', 'intake', 'inlet'],
 'front': ['フロント側', 'front'],
 'rear': ['リア側', 'rear']}

## Implement temperature control logic

### Subtask:
Develop code to take user input for target temperatures for each direction, using the direction mapping to handle ambiguous terms.


**Reasoning**:
Develop code to take user input for target temperatures for each direction, using the direction mapping to handle ambiguous terms.



In [11]:
import re, json
from typing import Dict, Any

# --- エイリアス定義（必要に応じて拡張してください） ---
direction_mapping = {
    "exhaust": ["排気側", "排気", "exhaust", "outlet"],
    "intake":  ["吸気側", "吸気", "intake", "inlet"],
    "front":   ["フロント側", "フロント", "front"],
    "rear":    ["リア側", "リア", "rear", "back"]
}
ALL_ALIASES = ["全部", "全て", "すべて", "all", "overall", "共通", "同じ"]

RE_NUMBER = r"(-?\d+(?:\.\d+)?)"

def _find_number_near(text: str, start_idx: int, window: int = 25):
    seg = text[start_idx:start_idx+window]
    m = re.search(rf"{RE_NUMBER}\s*(?:度|℃|°C|C)?", seg, flags=re.IGNORECASE)
    return float(m.group(1)) if m else None

def _parse_global_default(text: str):
    pat = r"(?:{}).*?{}\s*(?:度|℃|°C|C)?".format("|".join(map(re.escape, ALL_ALIASES)), RE_NUMBER)
    m = re.search(pat, text, flags=re.IGNORECASE | re.DOTALL)
    return float(m.group(1)) if m else None

def parse_free_text_to_dict(text: str, direction_mapping: Dict[str, list]) -> Dict[str, float]:
    """自由文から各部位温度を辞書化（数値: °C）。"""
    result = {}
    g = _parse_global_default(text)

    # 明示指定
    for key, aliases in direction_mapping.items():
        for alias in sorted(aliases, key=len, reverse=True):
            for m in re.finditer(re.escape(alias), text, flags=re.IGNORECASE):
                temp = _find_number_near(text, m.end(), window=25)
                if temp is not None:
                    result[key] = temp
                    break
            if key in result:
                break

    # グローバル既定で埋める
    if g is not None:
        for key in direction_mapping.keys():
            if key not in result:
                result[key] = g
    return result

def _ensure_all_keys(d: Dict[str, Any]) -> Dict[str, float]:
    """不足キーを問い合わせて補完（コンソール利用）。"""
    out = dict(d)
    for key, aliases in direction_mapping.items():
        if key not in out:
            terms = ", ".join(aliases)
            while True:
                v = input(f"{terms} の目標温度(℃)を入力してください: ")
                try:
                    out[key] = float(v)
                    break
                except ValueError:
                    print("数値で入力してください（例: 210）")
    return out

# ------------------ LLM 抽出（JSON専用） ------------------
def extract_with_llm_to_json(model, tokenizer, user_text: str) -> str:
    """
    LLMに JSON だけを出力させる。
    期待JSON: {"exhaust": 280, "intake": 220, "front": 210, "rear": 210}
    """
    system_rules = (
        "You are a strict JSON formatter. "
        "Output ONLY a minified JSON with four numeric °C values: "
        '{"exhaust": <float>, "intake": <float>, "front": <float>, "rear": <float>}. '
        "No comments, no prose, no markdown."
    )
    # Few-shot の例で形式を強化
    examples = [
        ("全部210、ただし排気は280", '{"exhaust":280,"intake":210,"front":210,"rear":210}'),
        ("front205C, rear 210, 吸気220, 排気280℃", '{"exhaust":280,"intake":220,"front":205,"rear":210}'),
    ]

    prompt = ""
    prompt += f"[SYSTEM]\n{system_rules}\n"
    for q, a in examples:
        prompt += f"[USER]\n{q}\n[ASSISTANT]\n{a}\n"
    prompt += f"[USER]\n{user_text}\n[ASSISTANT]\n"

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=128, do_sample=False, eos_token_id=tokenizer.eos_token_id)
    raw = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 末尾の最新回答のみ取り出し（念のため最後の { から } まで抽出）
    m = re.search(r"\{.*\}", raw, flags=re.DOTALL)
    if not m:
        raise ValueError("LLMからJSONが取得できませんでした")
    js = m.group(0)

    # 構文チェック & 型厳格化
    data = json.loads(js)
    # 必須キーの存在と数値化
    for k in ["exhaust", "intake", "front", "rear"]:
        if k not in data:
            raise ValueError(f"キー {k} が不足")
        data[k] = float(data[k])
    return json.dumps(data, ensure_ascii=False, separators=(",", ":"))

# ------------------ 統合I/O ------------------
def prompt_and_return_json(model, tokenizer) -> str:
    """
    1) 自由入力を受け取る
    2) LLMでJSON抽出を試みる
    3) 失敗時はローカル正規表現→不足分を対話で補完→JSON化
    4) JSON文字列（minified）を返す
    """
    print("どこの部位を何℃に設定しますか？自然文でまとめて入力OKです。")
    print("例: 『排気280℃、吸気220、front 205C、rearは210』 / 『全部210』 / 『全部210、ただし排気は280』")
    user_text = input(">> ")

    # まず LLM で厳格JSON抽出
    try:
        js = extract_with_llm_to_json(model, tokenizer, user_text)
        print(js)  # 仕様: 出力はJSON
        return js
    except Exception as e:
        # フォールバックへ
        parsed = parse_free_text_to_dict(user_text, direction_mapping)
        parsed = _ensure_all_keys(parsed)
        js = json.dumps(
            {
                "exhaust": float(parsed["exhaust"]),
                "intake":  float(parsed["intake"]),
                "front":   float(parsed["front"]),
                "rear":    float(parsed["rear"]),
            },
            ensure_ascii=False, separators=(",", ":")
        )
        print(js)
        return js

# ---- 実行例 ----
json_result = prompt_and_return_json(model, tokenizer)
print("JSON結果:", json_result)


どこの部位を何℃に設定しますか？自然文でまとめて入力OKです。
例: 『排気280℃、吸気220、front 205C、rearは210』 / 『全部210』 / 『全部210、ただし排気は280』
>> 吸気は210℃、排気は240℃、残りは吸気と同じ
フロント側, フロント, front の目標温度(℃)を入力してください: 190でお願い
数値で入力してください（例: 210）
フロント側, フロント, front の目標温度(℃)を入力してください: 190
リア側, リア, rear, back の目標温度(℃)を入力してください: 190
{"exhaust":240.0,"intake":210.0,"front":190.0,"rear":190.0}
JSON結果: {"exhaust":240.0,"intake":210.0,"front":190.0,"rear":190.0}


In [13]:
with open('input.json', 'w') as f:
    f.write(json_result)